In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/fish/

/content/drive/MyDrive/fish


In [ ]:
!unzip NA_Fish_Dataset.zip

In [ ]:
os.listdir("NA_Fish_Dataset")

['Black Sea Sprat',
 'Gilt Head Bream',
 'Horse Mackerel',
 'Red Mullet',
 'Red Sea Bream',
 'Sea Bass',
 'Shrimp',
 'Striped Red Mullet',
 'Trout',
 'anglerfish',
 'european sea sturgeon',
 'Red hand fish',
 'sakhalin sturgeon',
 'smalltooth fish',
 'tequils splitfin']

In [ ]:
!ls NA_Fish_Dataset/

 anglerfish		 'Red hand fish'       Shrimp
'Black Sea Sprat'	 'Red Mullet'	      'smalltooth fish'
'european sea sturgeon'  'Red Sea Bream'      'Striped Red Mullet'
'Gilt Head Bream'	 'sakhalin sturgeon'  'tequils splitfin'
'Horse Mackerel'	 'Sea Bass'	       Trout


In [ ]:
for i in os.listdir("NA_Fish_Dataset"):
  print(i,len(os.listdir("NA_Fish_Dataset/"+i)))

Black Sea Sprat 50
Gilt Head Bream 50
Horse Mackerel 50
Red Mullet 50
Red Sea Bream 50
Sea Bass 50
Shrimp 50
Striped Red Mullet 50
Trout 30
anglerfish 58
european sea sturgeon 50
Red hand fish 50
sakhalin sturgeon 54
smalltooth fish 51
tequils splitfin 56


In [ ]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass
for i in os.listdir("NA_Fish_Dataset"):
  try:
    os.mkdir("train/"+i)
    os.mkdir("test/"+i)
  except:
    pass
  for j in os.listdir("NA_Fish_Dataset/"+i)[:35]:
    os.rename("NA_Fish_Dataset/"+i+"/"+j,"train/"+i+"/"+j)
  for j in os.listdir("NA_Fish_Dataset/"+i)[:15]:
    os.rename("NA_Fish_Dataset/"+i+"/"+j,"test/"+i+"/"+j)

In [ ]:
def img_Data(dir_path,target_size,batch,class_lst,preprocessing,):
  if preprocessing:
    gen_object =ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object =ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                          target_size= target_size,
                                          batch_size=batch,
                                          class_mode='sparse',
                                          classes=class_lst,
                                          shuffle=True))

In [ ]:
train_data_gen =img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen =img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 500 images belonging to 15 classes.
Found 197 images belonging to 15 classes.


In [ ]:
base_model=tf.keras.applications.mobilenet_v2.MobileNetV2(
              input_shape=(224,224,3),
              alpha=1.0,
              include_top=False,
              weights='imagenet',
              input_tensor=None,
              pooling=None,
              classes=1000,
              classifier_activation='softmax')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable=False

In [ ]:
model=tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(1024,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
elst=callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck=callbacks.ModelCheckpoint('.mdl_wt.hdf5',save_best_only=True,monitor='val_loss',mode='min')

In [ ]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

Epoch 1/10
1/1 [==============================] - 68s 68s/step - loss: 6.8480 - accuracy: 0.0000e+00 - val_loss: 4.1723 - val_accuracy: 0.4924
Epoch 2/10
1/1 [==============================] - 54s 54s/step - loss: 4.0447 - accuracy: 0.4820 - val_loss: 2.1160 - val_accuracy: 0.5990
Epoch 3/10
1/1 [==============================] - 54s 54s/step - loss: 1.9404 - accuracy: 0.5940 - val_loss: 1.1644 - val_accuracy: 0.7970
Epoch 4/10
1/1 [==============================] - 54s 54s/step - loss: 0.9644 - accuracy: 0.8600 - val_loss: 0.7504 - val_accuracy: 0.8731
Epoch 5/10
1/1 [==============================] - 44s 44s/step - loss: 0.5455 - accuracy: 0.9420 - val_loss: 0.6333 - val_accuracy: 0.8274
Epoch 6/10
1/1 [==============================] - 43s 43s/step - loss: 0.3975 - accuracy: 0.9200 - val_loss: 0.4992 - val_accuracy: 0.8426
Epoch 7/10
1/1 [==============================] - 43s 43s/step - loss: 0.2594 - accuracy: 0.9580 - val_loss: 0.3979 - val_accuracy: 0.8731
Epoch 8/10
1/1 [=======